In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait,Select
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
import os

In [3]:
#Configuración básica:

LOGIN_URL= "https://gestion.reiteman.com/login"
VENTAS_URL= "https://gestion.reiteman.com/ventas-comerciales"
USERNAME = "Ivan"
PASSWORD = "123Ivan"
DOWNLOAD_DIR = r"C:\Users\Abraham\Desktop\Solo\Bootcamp\portfolio\PowerBI"

In [113]:
#EXTRACCIÓN COMERCIALES
# Configuración del navegador
options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=options)
wait = WebDriverWait(driver, 15)

try:
    # === 1. Login ===
    driver.get(LOGIN_URL)
    wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(USERNAME)
    driver.find_element(By.ID, "password").send_keys(PASSWORD)
    driver.find_element(By.ID, "kt_login_submit").click()

    # === 2. Ir a la página de ventas ===
    wait.until(EC.url_changes(LOGIN_URL))
    driver.get(VENTAS_URL)

    # === 3. Seleccionar "Comerciales" en el select ===
    select_element = wait.until(EC.presence_of_element_located((By.ID, "mostrar")))
    select = Select(select_element)
    select.select_by_value("N")  # "C" = Comerciales

    # === 4. Clic en botón "Buscar", con manejo de posibles bloqueos ===
    buscar_btn = wait.until(EC.visibility_of_element_located((By.ID, "kt_search")))
    driver.execute_script("arguments[0].scrollIntoView(true);", buscar_btn)
    try:
        buscar_btn.click()
    except:
        driver.execute_script("arguments[0].click();", buscar_btn)

    # === 5. Esperar a que aparezca la tabla ===
    wait.until(EC.presence_of_element_located((By.ID, "tabla_accionescomerciales")))

    # === 6. Clic en botón "Excel" ===
    excel_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "buttons-excel")))
    driver.execute_script("arguments[0].scrollIntoView(true);", excel_btn)
    try:
        excel_btn.click()
    except:
        driver.execute_script("arguments[0].click();", excel_btn)

    # === 7. Esperar descarga ===
    time.sleep(7)

finally:
    driver.quit()

print("✅ Archivo Excel descargado con éxito.")

✅ Archivo Excel descargado con éxito.


In [4]:
import shutil

In [5]:

def descargar_datos(tipo, nombre_archivo):
    # Configuración navegador
    options = webdriver.ChromeOptions()
    prefs = {
        "download.default_directory": DOWNLOAD_DIR,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)

    driver = webdriver.Chrome(options=options)
    wait = WebDriverWait(driver, 15)

    try:
        print(f"🔄 Descargando datos para tipo '{tipo}'...")

        # Login
        driver.get(LOGIN_URL)
        wait.until(EC.presence_of_element_located((By.ID, "username"))).send_keys(USERNAME)
        driver.find_element(By.ID, "password").send_keys(PASSWORD)
        driver.find_element(By.ID, "kt_login_submit").click()

        # Ir a página ventas
        wait.until(EC.url_changes(LOGIN_URL))
        driver.get(VENTAS_URL)

        # Elegir tipo
        select_element = wait.until(EC.presence_of_element_located((By.ID, "mostrar")))
        select = Select(select_element)
        select.select_by_value(tipo)

        # Buscar
        buscar_btn = wait.until(EC.visibility_of_element_located((By.ID, "kt_search")))
        driver.execute_script("arguments[0].scrollIntoView(true);", buscar_btn)
        try:
            buscar_btn.click()
        except:
            driver.execute_script("arguments[0].click();", buscar_btn)

        # Esperar tabla
        wait.until(EC.presence_of_element_located((By.ID, "tabla_accionescomerciales")))

        # Descargar Excel
        excel_btn = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "buttons-excel")))
        driver.execute_script("arguments[0].scrollIntoView(true);", excel_btn)
        try:
            excel_btn.click()
        except:
            driver.execute_script("arguments[0].click();", excel_btn)

        # Esperar descarga
        time.sleep(7)

        # Renombrar el archivo descargado más reciente
        archivos = [f for f in os.listdir(DOWNLOAD_DIR) if f.endswith(".xlsx")]
        if archivos:
            ultimo_archivo = max(
                [os.path.join(DOWNLOAD_DIR, f) for f in archivos],
                key=os.path.getctime
            )
            nuevo_path = os.path.join(DOWNLOAD_DIR, nombre_archivo)
            shutil.move(ultimo_archivo, nuevo_path)
            print(f"✅ Archivo guardado como: {nombre_archivo}")
        else:
            print("❌ No se encontró archivo .xlsx descargado.")

    finally:
        driver.quit()

# === Ejecutar para cada tipo ===
descargar_datos("T", "ventas_telemarketing.xlsx")
descargar_datos("C", "ventas_comerciales.xlsx")
descargar_datos("N", "ventas_tecnicos.xlsx")


🔄 Descargando datos para tipo 'T'...
✅ Archivo guardado como: ventas_telemarketing.xlsx
🔄 Descargando datos para tipo 'C'...
✅ Archivo guardado como: ventas_comerciales.xlsx
🔄 Descargando datos para tipo 'N'...
✅ Archivo guardado como: ventas_tecnicos.xlsx
